In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
with h5py.File('../build/debug/result.h5') as f:
    raw = f['dataset'][:]

In [ ]:
data = raw

In [ ]:
data.shape

In [ ]:
plt.plot(data[0, 0, :], data[0, 1, :], '.')
plt.gca().set_aspect(1)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data[0, 0, :], data[0, 1, :], data[0, 2, :])